In [ ]:
from xgboost import XGBRegressor, XGBClassifier
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.utils import shuffle
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from datetime import date, timedelta
import os

import neuralNets

def oneHotEncodeData3Classes(targets):
    j=0
    Y_val = np.zeros((targets.shape[0], 3))
    for j in range(targets.shape[0]):
        if targets[j] == 0:
            Y_val[j, 0] = 1
        elif targets[j] == 1:
            Y_val[j, 1] = 1
        elif targets[j] == 2:
            Y_val[j, 2] = 1
        else:
            print("something went wrong, new class", targets[j])
    return Y_val

In [ ]:
training_data = pd.read_csv("data/numerai_datasets_25.04.21/numerai_training_data.csv")

In [ ]:
validation_data = pd.read_csv("data/numerai_datasets_25.04.21/numerai_validation_data.csv") 

In [ ]:
X_train_3class = training_data[feature_cols]
Y_train_3class = training_data.target

Y_train_3class = Y_train_3class.replace(1, 0)
Y_train_3class = Y_train_3class.replace([0.25, 0.75], 1)
Y_train_3class = Y_train_3class.replace(0.5, 2)

X_train_3class = X_train_3class.to_numpy()
Y_train_3class = Y_train_3class.to_numpy()

X_train_3class, Y_train_3class = shuffle(X_train_3class, Y_train_3class)

X_train_3class, X_test_3class, Y_train_3class, Y_test_3class = train_test_split(X_train_3class, Y_train_3class, test_size = 0.3)

X_val_3class = validation_data[feature_cols]
Y_val_3class = validation_data.target

Y_val_3class = Y_val_3class.replace(1, 0)
Y_val_3class = Y_val_3class.replace([0.25, 0.75], 1)
Y_val_3class = Y_val_3class.replace(0.5, 2)

X_val_3class = X_val_3class.to_numpy()
Y_val_3class = Y_val_3class.to_numpy()

In [ ]:
feature_cols = training_data.columns[training_data.columns.str.startswith('feature')] 
features = ['feature_w', 'feature_con','feature_dex','feature_str','feature_cha','feature_int']
#print(feature_cols)
for feature in features:
    sub_features = training_data.columns[training_data.columns.str.startswith(feature)] 
    print(feature_cols.get_loc(sub_features[0]))
    #print(training_data.columns.get_loc(training_data.columns.str.startswith(feature)))

In [ ]:
X_train_3class_int = X_train_3class[:,0:12]
X_train_3class_cha = X_train_3class[:,12:98]
X_train_3class_str = X_train_3class[:,98:136]
X_train_3class_dex = X_train_3class[:,136:150]
X_train_3class_con = X_train_3class[:,150:264]
X_train_3class_wis = X_train_3class[:,264:310]

X_train = [X_train_3class_int, X_train_3class_cha, X_train_3class_str, X_train_3class_dex, X_train_3class_con, X_train_3class_wis]

X_test_3class_int = X_test_3class[:,0:12]
X_test_3class_cha = X_test_3class[:,12:98]
X_test_3class_str = X_test_3class[:,98:136]
X_test_3class_dex = X_test_3class[:,136:150]
X_test_3class_con = X_test_3class[:,150:264]
X_test_3class_wis = X_test_3class[:,264:310]
X_test = [X_test_3class_int, X_test_3class_cha, X_test_3class_str, X_test_3class_dex, X_test_3class_con, X_test_3class_wis]

X_val_3class_int = X_val_3class[:,0:12]
X_val_3class_cha = X_val_3class[:,12:98]
X_val_3class_str = X_val_3class[:,98:136]
X_val_3class_dex = X_val_3class[:,136:150]
X_val_3class_con = X_val_3class[:,150:264]
X_val_3class_wis = X_val_3class[:,264:310]
X_val = [X_val_3class_int, X_val_3class_cha, X_val_3class_str, X_val_3class_dex, X_val_3class_con, X_val_3class_wis]


In [ ]:
def defineNN_3classes(n_wisdom, n_con, n_dex, n_stra, n_cha, n_intel):
    activation = "relu"
    regularizationConst_l1 = 0.00000#3
    regularizationConst_l2 = 0.00000#3
    #size = 512
    W_input  = Input(shape=(n_wisdom,))
    W = Dense(32, activation=activation)(W_input)
    W = Dense(4, activation=activation)(W)
    W = Model(inputs=W_input, outputs=W)
    
    
    CO_input = Input(shape=(n_con,))
    CO = Dense(32, activation=activation)(CO_input)
    CO = Dense(4, activation=activation)(CO)
    CO = Model(inputs=CO_input, outputs=CO)
    
    D_input  = Input(shape=(n_dex,))
    D = Dense(32, activation=activation)(D_input)
    D = Dense(4, activation=activation)(D)
    D = Model(inputs=D_input, outputs=D)
    
    S_input  = Input(shape=(n_stra,))
    S = Dense(32, activation=activation)(S_input)
    S = Dense(4, activation=activation)(S)
    S = Model(inputs=S_input, outputs=S)
    
    CA_input = Input(shape=(n_cha,))
    CA = Dense(32, activation=activation)(CA_input)
    CA = Dense(4, activation=activation)(CA)
    CA = Model(inputs=CA_input, outputs=CA)
    
    I_input  = Input(shape=(n_intel,))    
    I = Dense(32, activation=activation)(I_input)
    I = Dense(4, activation=activation)(I)
    I = Model(inputs=I_input, outputs=I)
    
    combined = concatenate([W.output, CO.output])
    combined = concatenate([combined, D.output])
    combined = concatenate([combined, S.output])
    combined = concatenate([combined, CA.output])
    combined = concatenate([combined, I.output])
    
    X = Dense(16, activation=activation)(combined)
    X = Dense(8, activation=activation)(X)
    
    X = Dense(3, activation="softmax")(X)
    
    model = Model(inputs = combined, outputs = X)

    return model

In [ ]:
modelNN_3classes = defineNN_3classes(X_val_3class_int.shape[1], X_val_3class_cha.shape[1], X_val_3class_str.shape[1], X_val_3class_dex.shape[1], X_val_3class_con.shape[1], X_val_3class_wis.shape[1])
optAdam    = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.99)

modelNN_3classes.compile(optimizer='adam', loss='categorical_crossentropy', metrics='categorical_accuracy')

In [ ]:
era_count = 120
for i in range(era_count):
    

In [ ]:
data

In [ ]:
feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]

newFeatures = 0
for i in range(len(feature_cols)):
    feature1 = feature_cols[i]
    for j in range(len(feature_cols)- i - 1):
        feature2 = feature_cols[i+j+1]
        newFeatures += 1
        #data[feature1+feature2] = data[feature1].mul(data[feature2], 1)
        
        #print(data[feature1], data[feature2], data[feature1+feature2])
        #break
print(newFeatures)